In [115]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import ElasticNet
 
from sklearn import datasets, linear_model, kernel_ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut 

from sklearn.model_selection import cross_val_score



In [116]:
# Load training data CSV using Pandas
filename = 'train.csv'
data = pd.read_csv(filename, header=0)
# Separate training data into Y and X
array = data.values
Y = array[:, 1]
X = array[:, 2:7]

In [117]:
def phi(i, x):
    if(i == 0):
        return x
        
    elif(i == 1):
        return x**2
        
    elif(i == 2):
        return np.exp(x)
        
    elif(i == 3):
        return np.cos(x)
        
    else:
        return 1
    
def transformFeatures(X):
    
    XTransformed = []
    
    for i in range(4):        
        for j in range(5):
            XTransformed.append(phi(i, X[j]))
            
    XTransformed.append(1)
        
    return np.array(XTransformed)

def transformInput(X):
    
    XTransformed = []
    
    for row in X:
        XTransformed.append(transformFeatures(row))
        
    return np.array(XTransformed)

XTransformed = transformInput(X)
            
        
        

In [122]:
def RegressionWithCv(X, Y, model):
    kf = KFold(10, shuffle=False)#Evtl würde shuffle=True bessere Werte geben 
    err = 0
    i = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train, y_train) # we think fit resets the initial weigh vector. but not sure...
        y_eval = model.predict(X_test)
        err += mean_squared_error(y_test, y_eval)**0.5
        i += 1
    return err/i

r = np.arange(0.01, 1, 0.01)
lambdas = [*r]
r = np.arange(0,1,0.01)
radii = [*r]
r = np.arange(0,1,1)
degrees = [*r]

kernelString = "polynomial"

bestLambda = -1
bestRadius = -1
bestDegree = -1
rmbs = []
currentBest = float("inf")
for l in lambdas: 
    for h in radii:
        for d in degrees:
            #model = linear_model.Ridge(alpha = l, fit_intercept=False, random_state=3) 
            model = linear_model.Lasso(alpha = l, fit_intercept=False, normalize = False, random_state=3)
            #model = kernel_ridge.KernelRidge(alpha = l, kernel= kernelString, gamma = h, degree = d, coef0 = 1, kernel_params=None)
            #model = ElasticNet(alpha = l, l1_ratio = h)
            tmpError = RegressionWithCv(XTransformed, Y, model)
            if (tmpError < currentBest):
                currentBest = tmpError
                bestLambda = l
                bestRadius = h
                bestDegree = d
            
            rmbs.append(tmpError)
    

print("best error:")
print(currentBest)
  
print("best lambda:")
print(bestLambda)

print("best Radius:")
print(bestRadius)

print("best degree:")
print(bestDegree)


#model = linear_model.Ridge(alpha = bestLambda, fit_intercept=False, random_state=3)model = linear_model.Lasso(alpha = bestLambda, fit_intercept=False, normalize = False, tol = 0.000001, random_state=3)
#model = kernel_ridge.KernelRidge(alpha = bestLambda, kernel= kernelString, gamma = bestRadius, degree = bestDegree, coef0 = 1, kernel_params=None)
##model = ElasticNet(alpha = bestLambda, l1_ratio = bestRadius)
model.fit(XTransformed, Y)


result = model.coef_
#result = np.dot(XTransformed.transpose(), model.dual_coef_)
print(result)


C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


best error:
10.519515752923173
best lambda:
0.09999999999999999
best Radius:
0.0
best degree:
0
[-0.          1.14167419 -1.11712499  5.13713543 -1.73383613 -0.
  0.         -0.          1.23663273 -0.         -0.2212895  -3.17906621
 -0.02636211  0.00753895 -0.22968244  0.         -0.          0.
 -0.          0.         -0.        ]


In [113]:
filename = 'result.csv'
final_result = pd.DataFrame(result)
final_result.to_csv(filename, header=False, index=False) 